In [1]:
#api_key = "Your Api key Here" 

In [2]:
from apiclient.discovery import build
youtube = build('youtube', 'v3', developerKey=api_key)

In [3]:
import pandas as pd

In [7]:
# Loading all video ids list from a channel
df = pd.read_csv('wordaffairs_yotube_videos_ids.csv')

In [8]:
df

,0
0,9Gg3oC_vGHA
1,-YQoaGSxTcU
2,FDTdNRCyTPM
3,rYKCObf-mwc
4,focbd6NLZXU
...,...
757,HoOJGfgaOQY
758,YjKHAgFek7w
759,Ph3kslBQyIM
760,ZinLUkbL9bk


In [9]:
# converting video ids into a list data type.
ID = df["0"].tolist()
ID

['9Gg3oC_vGHA',
 '-YQoaGSxTcU',
 'FDTdNRCyTPM',
 'rYKCObf-mwc',
 'focbd6NLZXU',
 'ZYWXNtuPBsw',
 'ddlSmy2usHM',
 'HDOog5kgCd8',
 'osH-8FUJ-LA',
 '0H7tm9Ve5Xc',
 '679RmQ3S5vY',
 'XxvBM0wRvEo',
 'hCAGi41cBd8',
 'NCbdMxZXvQ8',
 '9CXUJHXeTM4',
 'MbslVNumfno',
 'wl3NVNxD3tY',
 'O6C853RVwGE',
 '6YIsBj4K3Mo',
 'bz05ARwQtB0',
 'KSAFmoHkagQ',
 'ysQqGbVGOhI',
 'nMRwThluei0',
 'Vv8UqqYO9Lg',
 'vvABFPkm6wg',
 'gPmi7CFAw_E',
 'LL81Anw9MEg',
 'zjFLUwfgt3E',
 'd5QN8LDQw58',
 'aCrdw_it7u8',
 '3Spj8rVM1xs',
 'qpzbvjT8wBI',
 'd9h1JIQaHEQ',
 'I9SIeZJVBiA',
 'VtejZ0PM7vY',
 'J057biMm3u0',
 'yofpG4k3FEs',
 'gKNw-uWcCBA',
 '8pd1zMQQIVQ',
 'FhWKeSEtIqc',
 'l7mXpMdfSuA',
 'Nks4osKm6aQ',
 '_sd5-LgUDIQ',
 '0tPYB0-fj1g',
 'jYn61XfmSCo',
 '0CpSQ6sNJZ4',
 'Awg1WAzvyaY',
 '7P_0mRCV4ds',
 '0BRvnGO0Bp8',
 '6dhkscyxI8Q',
 'AGiO1vm65Ew',
 'rsZO4MvFLgI',
 'gdcH_u5e9D4',
 'ixJDulEtrEk',
 'jKPm1F3bapY',
 'aZHVVsZIaNo',
 'qrALkh3iB0g',
 '9zJtIAKUMto',
 'XyR1ebo-55Y',
 '4XjpqO5Xq_c',
 '04KTITZCI-4',
 'HrSeXC04GNI',
 'YdR1f-

In [4]:
#ID = ["zeZJeBCqUo0"]

In [5]:
box = [['Name', 'Comment', 'Time', 'Likes', 'Reply Count']]
n=0

def scrape_comments_with_replies(n):
    data = youtube.commentThreads().list(part='snippet', videoId=ID[n], maxResults='100', textFormat="plainText").execute()

    for i in data["items"]:

        name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
        comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
        published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
        likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
        replies = i["snippet"]['totalReplyCount']

        box.append([name, comment, published_at, likes, replies])

        totalReplyCount = i["snippet"]['totalReplyCount']

        if totalReplyCount > 0:

            parent = i["snippet"]['topLevelComment']["id"]

            data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent,
                                            textFormat="plainText").execute()

            for i in data2["items"]:
                name = i["snippet"]["authorDisplayName"]
                comment = i["snippet"]["textDisplay"]
                published_at = i["snippet"]['publishedAt']
                likes = i["snippet"]['likeCount']
                replies = ""

                box.append([name, comment, published_at, likes, replies])

    while ("nextPageToken" in data):

        data = youtube.commentThreads().list(part='snippet', videoId=ID[n], pageToken=data["nextPageToken"],
                                             maxResults='100', textFormat="plainText").execute()

        for i in data["items"]:
            name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
            comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
            published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
            likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
            replies = i["snippet"]['totalReplyCount']

            box.append([name, comment, published_at, likes, replies])

            totalReplyCount = i["snippet"]['totalReplyCount']

            if totalReplyCount > 0:

                parent = i["snippet"]['topLevelComment']["id"]

                data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent,
                                                textFormat="plainText").execute()

                for i in data2["items"]:
                    name = i["snippet"]["authorDisplayName"]
                    comment = i["snippet"]["textDisplay"]
                    published_at = i["snippet"]['publishedAt']
                    likes = i["snippet"]['likeCount']
                    replies = ''

                    box.append([name, comment, published_at, likes, replies])

    df = pd.DataFrame({'Name': [i[0] for i in box], 'Comment': [i[1] for i in box], 'Time': [i[2] for i in box],
                       'Likes': [i[3] for i in box], 'Reply Count': [i[4] for i in box]})
    df.to_csv('youtube_comments.csv', index=False, header=False)

while(n<len(ID)):
  scrape_comments_with_replies(n)
  n=n+1

In [ ]:
# It  saved comments into  'youtube_comments.csv' file.